In [48]:
import pandas as pd
import numpy as np
from pandas import Series, DataFrame

## 데이터 변형

#### 1. removing duplicates
- 중복된 row를 식별하고 이를 제거하는 방법을 학습한다.
- df.duplicated() : df.drop_duplicates()의 하위 개념으로 함수 적용 시, 중복인지 아닌지 여부를 boolean Series로 반환.
- subset : column label 혹은 sequence of labels. 중복을 식별할 열을 지정.
- keep
- inplace

In [ ]:
df.drop_duplicates = df[~df.duplicated()] : 같은 의미. why? boolean Series에 의한 indexing

In [ ]:
df.drop_duplicates(['k1']) # 1개의 열을 고르더라도 subset으로 다루기 때문에 여러 열을 사용하듯이 지정해야함.

#### 2. transforming, mapping

#### mapping
- 'A' 열의 설명을 위한 새로운 column을 추가하려고 할 때.
- 'A'의 각 열의 key을 값으로하는 열을 추가하고 싶을 때. map 함수 사용
- map : vector(Series)의 각 성분별 함수 결과 또는 각 성분을 key로하는 value를 리턴한다. df에서는 사용불가

ex) data['col1'] = data['A'].map(str, lower).map(dict) # dict의 key는 'A'의 각 열과 동일하다.

#### 3. replace value
- 결측치 대체
- 

In [1]:
data = Series([1, -999, 3, -999, 5])
data

NameError: name 'Series' is not defined

In [ ]:
data.replace(-999, np.nan)
data.replace([-999, 5], [np.nan, 0])

#### 4. renaming axis indexes

#### 5.Discretization and binning(개별화와 양자화)
연속된 data는 종종:
- 개별로 분할하거나
- 분석을 위한 그룹(bin)으로 나눈다.

pd.cut()
- bin으로 나눌 구간은 ( , ]으로 정의되므로 첫 항은 빠진다. 따라서 일반적으로 +- 5% 정도 범위를 늘려서 계산한다.
- label을 지정하면 각 구간에 이름도 넣어줄 수 있다.
- 주로 설문조사를 컴퓨터에 입력할 때 사용. ex) 이를 coding이라고 함. 
- 구간을 나눈 요소들을 들어오는 데이터에 적용해서 code로 나타냄.

pd.qcut()
- 그룹별로 속한 개체 수가 다르지만, 동일한 크기의 그룹수를 지정하고자 할 때 사용.
- 백분율기반의 bin을 생산하므로, 항상 동일한 크기의 그룹을 보장한다.

#### 6. filtering outliers
- df.describe()

In [1]:
# 통계량 boolean indexing 적용해보기
ex) data[['col1', 'col2']][:5][True, True, False, False, False] 

SyntaxError: invalid syntax (<ipython-input-1-6e1a79654c54>, line 2)

In [ ]:
# Series 적용해보기
col = data[3]
col[np.abs(col) > 3] 

- df.any

In [4]:
arr = np.array([0, -5, 2, 7])
arr > 0

array([False, False,  True,  True])

In [ ]:
l = [0, -5, 2, 7]

b = list(map(lambda x: x>0, l)) # list boolean은 이런식으로...?
any(b) # 1개라도 True면 True

In [12]:
all(b) # 전부 True가 아니라면 False

False

In [ ]:
(np.abs(data) > 3).any(axis = 0) # np.abs(data) > 3는 df으로 boolean indexing 되서 반환된다.

In [ ]:
data[(np.abs(data) > 3).any(axis = 0)] # boolean indexing으로 True인 데이터만 df으로 반환.

In [ ]:
data[np.abs(data) > 3] = np.sign(data) * 3 # 왼쪽의 data는 data size가 유지됨.
data.describe()

#### 7. Permutation and random sampling(치환과 임의 샘플링)
비복원 추출  
np.permutation()
- 중복된 row는 없음.
- 주머니에서 'a'구슬을 뽑으면 그 구슬은 다시 집어넣지 않는다.
- row를 임의의 순서로 재배열하거나, 임의 샘플링하는 방법을 알아본다.
- 순서를 바꾸고 싶은 만큼의 길이를 np.permutation 함수에 넘기면 임의 순서의 정수배열을 얻을 수 있다.

In [16]:
df = DataFrame(np.arange(5 * 4).reshape((5,4)))
sampler = np.random.permutation(5)
sampler

array([4, 0, 3, 1, 2])

In [20]:
# 여기서 반만 train data 나머지는 test data로 쓰려고 한다.
# 주로 사용하는 예시
# titanic data 다 합친거를 다시 7대3으로 나눠보자.
i = sampler.size * 0.5
i = round(i)
train_idx = sampler[:i]
val_idx = sampler[i:]
print(train_idx)
print(val_idx)

[4 0]
[3 1 2]


In [ ]:
train_df = df.iloc[train_idx]
val_df = df.iloc[val_idx]

In [23]:
df.take(sampler) # sampler에 의해 섞인 df 

,0,1,2,3
4,16,17,18,19
0,0,1,2,3
3,12,13,14,15
1,4,5,6,7
2,8,9,10,11


In [25]:
df.iloc[sampler] # 위와 동일한 연산

,0,1,2,3
4,16,17,18,19
0,0,1,2,3
3,12,13,14,15
1,4,5,6,7
2,8,9,10,11


복원추출
- 중복된 row도 포함.
- np.random.randint 사용.
- 주머니에서 구슬뽑고 다시 구슬 넣음

np.random.randint()
- [low, high] 사이의 임의의 정수 size만큼 리턴

#### 8. Computing indicator and dummy variables(표시자와 범주형 변수)
- 예측 model에는 전부 숫자만 들어갈 수 있음.
- 이항 : 남녀, P/N, 동전 앞/뒤
- 다항 : 서열(범주+순서에 의미가 있는 것), Categorical

one-hot encode
- sparse하다? 대부분이 0이고 1이 드물다.
- sparse matrix : df를 조작하여 만들고, one-hot encoding하여 숫자로 풀어서 다시 만든 것.
- 예측 model에는 pandas로 조작한 df가 아니라 one-hot encoding하여 전부 숫자로 변환한 것을 넣는다.
- 정보가 유실되지 않는다.

서열범주
- one-hot encoding : data가 유실되지 않음. but, data가 무거워짐.
- 등간 숫자 : 의미 있는 순서를 1,2,3으로 만듬(숫자의 weight를 조절할 수 있음. ex) 10, 30, 50. / but, data가 유실될 수도 있음

Categorical
- ex) 고기의 육류별 종류를 육해공으로 압축.
- 분석 목적에 따라 압축시키는 방법은 전부 다름.

#### pd.get_dummies()
- 범수형 변수(지시자)를 one-hot encoding(더미변수)형식으로 변경하는 것이다.
- drop_first : 한 차원을 더 축소시키는 방법. 주로 R에서 사용
- add_prefix('name_') : 더미 이름 붙여주기

In [ ]:
# df + dummies 
pd.get_dummie(df, key = ['key'])

#### movielens 영화 데이터의 더미변수 생성
- 구분자가 2글자 이상이거나, 정규 표현식인 경우. c parsing engine이 동작하지 않아 경고메세지 나옴. engine=python으로 해주셈

In [ ]:
# 장르를 통해 one-hot encoding을 실시해보자.

genre_iter = (set(x.split('|')) for x in movies.genres) # 장르 데이터를 집합으로 만듦. 
# 이는 generator로써 값을 가진게 아니라 iterator를 동작하게 하는 역할
genre_iter # 데이터를 담고 있지 않음.

In [27]:
# 범주를 나눌 때 효과적인 code
genre = sorted(set.union(*genre_iter))

- set.union 함수는 genre_iter을 매 반복마다 합집합 시킴. 즉, data를 늘리지않고 unique한 요소들만 계속 추가시킴.
- sorted 내장함수는 iterable의 각 속성을 정렬해서 list로 반환한다.

___
- generator, iterator, iterable에 대해 알아보자.

f(v) for v in container : iterator. 값을 가진게 아니라 일정하게 찍어내기만함. 불변(immutable)
- list comprehension : [f(v) for v in container]
- generator : (f(v) for v in container), tuple처럼 보이지만 generator ^ ^
- set : {f(v) for v in container}
- dict : {}

___

In [ ]:
dummies = DataFrame(np.zeros((len(movies), len(genres))), columns=genres)

In [ ]:
for i, gen in enumerate(movies, genres):
    dummies.loc[i, gen.split('|')] = 1

- df에서 genre 열을 삭제하고, dummies로 만든 새로운 column을 결합하자.

## 문자열 다루기
str 클래스의 객체인 문자열 내장 메소드:

- split
- strip : 앞뒤 공백 제거
- join
- index(',') : , index의 위치를 찾음.
- find(',') : , 의 값을 찾음.
- count(',') : , 의 개수 찾음. 
- replace

## String manipulation

### String object methods

In [6]:
val = 'a,b,  guido'
val.split(',')

['a', 'b', '  guido']

In [5]:
"abc \t\r\n".strip()

'abc'

In [7]:
pieces = [x.strip() for x in val.split(',')]
pieces

['a', 'b', 'guido']

In [8]:
first, second, third = pieces
first + '::' + second + '::' + third

'a::b::guido'

In [9]:
'::'.join(pieces) 

'a::b::guido'

In [10]:
'guido' in val

True

In [13]:
val.index(',')

1

In [14]:
val.find(':') # 없으면 -1

-1

In [18]:
val.index(':') # 없으면 error

ValueError: substring not found

In [17]:
val.count(',')

2

In [19]:
val.replace(',', '::')

'a::b::  guido'

In [20]:
val.replace(',', '')

'ab  guido'

### Regular expressions(정규 표현식)
- 복잡한 문자열을 다룰 때 사용함.
- re는 문자열 패턴을 다루는 module

In [28]:
import re
text = "foo    bar\t baz  \tqux"
re.split('\s+', text)

['foo', 'bar', 'baz', 'qux']

In [30]:
regex = re.compile('\s+') # 자주 사용하는 표현은 re module의 compile 함수를 이용한다.
regex.split(text)

['foo', 'bar', 'baz', 'qux']

In [31]:
regex.findall(text)

['    ', '\t ', '  \t']

In [32]:
text = """Dave dave@google.com
Steve steve@gmail.com
Rob rob@gmail.com
Ryan ryan@yahoo.com
"""
pattern = r'[A-Z0-9._%+-]+@[A-Z0-9.-]+\.[A-Z]{2,4}' # r이 있으므로 \ 없이도 특수문자 인식가능.
# 이메일을 찾아내는 pattern

# re.IGNORECASE makes the regex case-insensitive
regex = re.compile(pattern, flags=re.IGNORECASE)

In [34]:
type(regex)

re.Pattern

In [39]:
regex.findall(text)

['dave@google.com', 'steve@gmail.com', 'rob@gmail.com', 'ryan@yahoo.com']

#### re.Match 객체
- 그룹 내용과 매칭이 어디서 일어 났는지에 대한 인덱스를 제공한다.
- 검색결과를 분석할 때 유용하다.

In [40]:
m = regex.search(text)
m

<re.Match object; span=(5, 20), match='dave@google.com'>

In [43]:
text[m.start():m.end()]

'dave@google.com'

In [ ]:
# 아래는 text의 첫 부분이 regex와 일치하지 않아 못 찾음.

In [44]:
print(regex.match(text))

None


In [45]:
print(regex.sub('REDACTED', text))

Dave REDACTED
Steve REDACTED
Rob REDACTED
Ryan REDACTED



In [ ]:
pattern = r'([A-Z0-9._%+-]+)@([A-Z0-9.-]+)\.([A-Z]{2,4})'
regex = re.compile(pattern, flags=re.IGNORECASE)

In [ ]:
m = regex.match('wesm@bright.net')
m.groups()

In [ ]:
regex.findall(text)

In [ ]:
print(regex.sub(r'Username: \1, Domain: \2, Suffix: \3', text))

In [ ]:
regex = re.compile(r"""
    (?P<username>[A-Z0-9._%+-]+)
    @
    (?P<domain>[A-Z0-9.-]+)
    \.
    (?P<suffix>[A-Z]{2,4})""", flags=re.IGNORECASE|re.VERBOSE)

In [ ]:
m = regex.match('wesm@bright.net')
m.groupdict()

### Vectorized string functions in pandas

- Series.str()으로 정규표현식 사용 가능.
- 원하는 col을 정규표현식으로 적용한다.
- 못찾으면 NaN을 반환하는데, 이는 나중에 결측치 제거를 통해 없애면 된다.
- Series.str.match()
- Series.str.contain()
- Series.str.findall()
- Series.str.get_dummies()

In [49]:
data = {'Dave': 'dave@google.com', 'Steve': 'steve@gmail.com',
        'Rob': 'rob@gmail.com', 'Wes': np.nan}
data = Series(data)

In [50]:
data

Dave     dave@google.com
Steve    steve@gmail.com
Rob        rob@gmail.com
Wes                  NaN
dtype: object

In [51]:
data.isnull()

Dave     False
Steve    False
Rob      False
Wes       True
dtype: bool

In [52]:
data.str.contains('gmail')

Dave     False
Steve     True
Rob       True
Wes        NaN
dtype: object

In [53]:
pattern

'[A-Z0-9._%+-]+@[A-Z0-9.-]+\\.[A-Z]{2,4}'

In [54]:
data.str.findall(pattern, flags=re.IGNORECASE)

Dave     [dave@google.com]
Steve    [steve@gmail.com]
Rob        [rob@gmail.com]
Wes                    NaN
dtype: object

In [56]:
matches = data.str.match(pattern, flags=re.IGNORECASE) # match는 첫 부분부터 맞춰봄.
matches

Dave     True
Steve    True
Rob      True
Wes       NaN
dtype: object

In [57]:
matches.str.get(1)

Dave    NaN
Steve   NaN
Rob     NaN
Wes     NaN
dtype: float64

In [58]:
matches.str[0]

Dave    NaN
Steve   NaN
Rob     NaN
Wes     NaN
dtype: float64

In [60]:
data.str.len()

Dave     15.0
Steve    15.0
Rob      13.0
Wes       NaN
dtype: float64

In [59]:
data.str[:5]

Dave     dave@
Steve    steve
Rob      rob@g
Wes        NaN
dtype: object